# Introduction to Remote Sensing and Geospatial analysis with Python

The goal of these notebooks is to introduce some python geospatial analysis tools and make us familiar with it.

In this notebook we will start installing the main libraries we will be using and introduce some basic concepts and commands, and also create an interactive map.

The following cell will install Earth Engine API and GeeHydro Library. Earth Engine API, or EE for short, is a Google API that allows us to acquire some satellite data and use a lot of useful tools to process Remote Sensing data. First, you have to sing up on this <a href="https://earthengine.google.com/">link</a> and then you will be able to continue with this notebook.

In [ ]:
!pip install earthengine-api
!pip install geehydro

In [27]:
import ee
import folium
import geehydro

Now, you have to authenticate your Earth Engine account. Just run the cell below and Google will tell you what to do.

In [ ]:
ee.Authenticate()
ee.Initialize()

## Create an Interactive map
First, we will initialize an interactive map using <a href="https://python-visualization.github.io/folium/">Folium</a>. Folium is an data visualization library that allows us to create maps, add markers, see the coordinates, add popups and aggregate multiple data layers in one map, while we can enable and disable the layers as it creates an interactive map, using <a href="https://leafletjs.com/">Leaflet</a> javascript maps.

In [42]:
# here we define the center location where the map will zoom when initialized, and the zoom it will have
_map = folium.Map(location=[-0.784176, -60.084085], zoom_start=6)
# a geehydro function so we can see a google satellite view as a basemap
_map.setOptions('HYBRID')

## Calculating NDVI

Now, some maths. NDVI is a basic concept on Remote Sensing, largely used on application focused on vegetation. NDVI stands for Normalized Difference Vegetation Index and utilizes the NIR and Red bands of the images. It is an indicator of quantity and quality of vegetation.

It is calculated this way:
## $\frac{NIR - Red}{NIR + Red}$

In the cell below, we will start using Earth Engine library. This library has a set of table datasets that we can use as Feature Collection. As example we are using USDOS/LSIB_SIMPLE/2017 table dataset. It is a dataset with polygons of countries' boundaries that we can use to select the boundaries one or more countries as we wish. This is one example of how helpful EE can be, it is very easy to select a country's boundaries and select data inside those boundaries. After getting the dataset with countries, we select the one we want, which is Brazil.

Then, we select data from the Landsat 7 satellite from two different periods in time. The first period from 1999 to 2003 and the second from 2008 to 2012, and it creates and composite of all those years. As easy as it, we have two 5-year composites of the whole world that we can manipulate. Next we apply the concept we just learn and calculate the NDVI using two different ways. The first and "hard" way is to use some methods that calculates the NDVI just like the equation described above. And the easy way, we just call a function selecting the bands we want to use to calculate it.

And in the last command, we crop the image to Brazil's boundaries and add the NDVI from 2008-2012 composite to our interactive map.

In [45]:
# Load country data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
brazil = countries.filter(ee.Filter.eq('country_na', 'Brazil'))

# Load two 5-year landsat 7 composites
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Computing NDVI - hard way
ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3')) \
                        .divide(landsat1999.select('B4')).add(landsat1999.select('B3'))

# Computing NDVI - easy way
ndvi2008 = landsat2008.normalizedDifference(['B4','B3'])

_map.addLayer(ndvi2008.clip(brazil),
              {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']},
              'NDVI')

In [46]:
# Show NDVI
_map.setControlVisibility(layerControl = True, fullscreenControl = True, latLngPopup = True)
_map